In [1]:
import os
import json

def load_json_files(folder_path):
    data = []
    for folder_name in os.listdir(folder_path):
        folder_full_path = os.path.join(folder_path, folder_name)
        if os.path.isdir(folder_full_path):
            print(folder_full_path)
            files = os.listdir(folder_full_path)

            for file_name in files:
                if file_name.endswith('.json'):
                    file_path = os.path.join(folder_full_path, file_name)
                    dataset = {
                        'model' : folder_name.replace("_image" , ""),
                        'dataset' : file_name.replace("_results.json" , ""),
                        'results' : None
                    }

                    with open(file_path, 'r') as f:
                        dataset['results'] = json.load(f)

                    data.append(dataset)
    return data

# Load all JSON files in the 'res' folder
folder_path = './results2/'
data = load_json_files(folder_path)

for d in data:
    print(d['model'], d['dataset'], len(d['results']))
    print(d['results'][0])


./results2/TranSalNetRes_finetune_true_threshold_0.0
./results2/TranSalNetRes_finetune_true_threshold_0.5
./results2/TranSalNetRes_finetune_true_threshold_0.8
./results2/TranSalNetRes_finetune_true_threshold_0.9
TranSalNetRes_finetune_true_threshold_0.0 mit1003 1003
{'filename': 'i05june05_static_street_boston_p1010764.jpeg', 'path': 'C:/Users/lelon/Documents/Dataset/MIT1003/images/images', 'metrics': {'GT': {'NSS': 0.6779999732971191, 'CC': 1.0, 'KLD': 1.2243, 'SIM': 1.0}, 'saliency': {'NSS': 0.8418999910354614, 'CC': 0.7472, 'KLD': 0.5648, 'SIM': 0.6104}, 'rarity': {'NSS': 0.06340000033378601, 'CC': 0.2301, 'KLD': 1.5259, 'SIM': 0.7469}, 'saliency_Add': {'NSS': 0.6406000256538391, 'CC': 0.6566, 'KLD': 0.9478, 'SIM': 0.8066}, 'saliency_Itti': {'NSS': 0.65829998254776, 'CC': 0.6674, 'KLD': 0.9276, 'SIM': 0.8042}}}
TranSalNetRes_finetune_true_threshold_0.5 mit1003 1003
{'filename': 'i05june05_static_street_boston_p1010764.jpeg', 'path': 'C:/Users/lelon/Documents/Dataset/MIT1003/images/i

In [2]:
import numpy as np

# def calculate_mean(data,key):
#     msrts, msrbs = [], []
#     for item in data:
#         msrts.append(item['metrics'][key]['msrt'])
#         msrbs.append(item['metrics'][key]['msrb'])  
#     msrts = np.asarray(msrts)
#     msrbs = np.asarray(msrbs)
#     mean_msrt = msrts.sum() / len(msrts)
#     mean_msrb = msrbs.sum() / len(msrbs)
#     return float(round(mean_msrt,2)), float(round(mean_msrb,2))


def calculate_mean(data,key,mkey):
    m= []
    for item in data:
        if item['metrics'][key][mkey] is None or np.isnan(item['metrics'][key][mkey]):
            continue
        m.append(item['metrics'][key][mkey])

    if len(m) == 0:
        return None
    
    m = np.asarray(m)
    m = m.sum() / len(m)
    return float(round(m,2))


results = []
for d in data:
    # get list of all keys resulst
    print(d)
    sal_keys = list(d['results'][0]['metrics'].keys())


    for key in sal_keys:

        result= {
            'model' : d['model'],
            'dataset' : d['dataset'],
            'key' : key,
            'metrics' :  {
            }
        }   

        metrics_keys = list(d['results'][0]['metrics'][key].keys())
        for mkey in metrics_keys:
            mean__ = calculate_mean(d['results'],key,mkey)
            if mean__ is None:
                continue
            result['metrics'][mkey] = mean__


        

        print(result)

        results.append(result)

    # break

{'model': 'TranSalNetRes_finetune_true_threshold_0.0', 'dataset': 'mit1003', 'results': [{'filename': 'i05june05_static_street_boston_p1010764.jpeg', 'path': 'C:/Users/lelon/Documents/Dataset/MIT1003/images/images', 'metrics': {'GT': {'NSS': 0.6779999732971191, 'CC': 1.0, 'KLD': 1.2243, 'SIM': 1.0}, 'saliency': {'NSS': 0.8418999910354614, 'CC': 0.7472, 'KLD': 0.5648, 'SIM': 0.6104}, 'rarity': {'NSS': 0.06340000033378601, 'CC': 0.2301, 'KLD': 1.5259, 'SIM': 0.7469}, 'saliency_Add': {'NSS': 0.6406000256538391, 'CC': 0.6566, 'KLD': 0.9478, 'SIM': 0.8066}, 'saliency_Itti': {'NSS': 0.65829998254776, 'CC': 0.6674, 'KLD': 0.9276, 'SIM': 0.8042}}}, {'filename': 'i05june05_static_street_boston_p1010785.jpeg', 'path': 'C:/Users/lelon/Documents/Dataset/MIT1003/images/images', 'metrics': {'GT': {'NSS': 0.713100016117096, 'CC': 1.0, 'KLD': 1.1641, 'SIM': 1.0}, 'saliency': {'NSS': 0.6118999719619751, 'CC': 0.6465, 'KLD': 1.5133, 'SIM': 0.5385}, 'rarity': {'NSS': 0.3831999897956848, 'CC': 0.3774, 'KL

In [3]:
from IPython.display import display, HTML
# get list of datasets from results
datasets_name = list(set([result['dataset'] for result in results]))
datasets_results = {dataset : [] for dataset in datasets_name}

print(datasets_results)

for result in results:
    datasets_results[result['dataset']].append(result)

import pandas as pd
for dataset, data in datasets_results.items():
    print(dataset)

    table_data = {
        'Models': [],
        'Type': [],
        # 'MSRT': msrts,
        # 'MSRB': msrbs
    }

    metrics= []
    for res in data:
        name = res['model'] + " " + res['key']

        table_data['Models'].append(res['model'])
        table_data['Type'].append(res['key'])

        for mk, mv in res['metrics'].items():
            if mk not in table_data:
                metrics.append(mk)
                table_data[mk] = []
            table_data[mk].append(mv)
    # show tables results of names msrts and msrbs

    df = pd.DataFrame(table_data)
    styled_df = df.style.background_gradient(subset=metrics, cmap='coolwarm')
    display(styled_df)




{'mit1003': []}
mit1003


,Models,Type,NSS,CC,KLD,SIM
0,TranSalNetRes_finetune_true_threshold_0.0,GT,0.890000,1.000000,1.550000,1.000000
1,TranSalNetRes_finetune_true_threshold_0.0,saliency,0.980000,0.660000,0.930000,0.560000
2,TranSalNetRes_finetune_true_threshold_0.0,rarity,0.450000,0.350000,1.630000,0.710000
3,TranSalNetRes_finetune_true_threshold_0.0,saliency_Add,0.880000,0.620000,1.120000,0.720000
4,TranSalNetRes_finetune_true_threshold_0.0,saliency_Itti,0.890000,0.620000,1.100000,0.720000
5,TranSalNetRes_finetune_true_threshold_0.5,GT,0.890000,1.000000,1.550000,1.000000
6,TranSalNetRes_finetune_true_threshold_0.5,saliency,0.980000,0.660000,0.930000,0.560000
7,TranSalNetRes_finetune_true_threshold_0.5,rarity,0.670000,0.450000,1.440000,0.730000
8,TranSalNetRes_finetune_true_threshold_0.5,saliency_Add,0.920000,0.620000,1.090000,0.720000
9,TranSalNetRes_finetune_true_threshold_0.5,saliency_Itti,0.930000,0.620000,1.070000,0.710000
